In [ ]:
#| default_exp connect

# connect -- A pickleball database connection 

> connect to the pklmart database

In [72]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
import psycopg2
import pandas as pd

In [ ]:
#| export
from configparser import ConfigParser

def config(filename="../database.ini", section="postgresql"):
    """Reads the database.ini file and returns the connection parameters as a dictionary.
    Assumes the ini file is in the parent directory"""
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception(
            "Section {0} not found in the {1} file".format(section, filename)
        )

    return db

In [ ]:
show_doc(config)

---

### config

>      config (filename='../database.ini', section='postgresql')

Reads the database.ini file and returns the connection parameters as a dictionary.
Assumes the ini file is in the parent directory

You will now want to set up a `database.ini` file. This will contain the sensitive information needed to connect to the databse. To avoid this information being public, be sure to add `*.ini` to your `.gitignore`. Once completed, it should look something like this:

#### **`database.ini`**
```
[postgresql]
database=DATABASE
host=HOST
user=USERNAME
password=PASSWORD
port=PORT
```
where you replace the uppercase values with the appropriate information.

We can now use the config function to get the appropriate information:

In [73]:
params = config()

We can now connect to the database using a `DbConnection` object

In [74]:
class DbConnection:
    "Class to create a connection to the database"
    def __init__(self, 
                params:dict):
        #todo add checks for params type and values
        if not isinstance(params, dict):
            raise TypeError(f"params must be a dictionary")
        self.params = params 
        self.conn = None
    def __str__(self): return f"connection object"
    __repr__ = __str__

    def connect(self):
        try:
            self.conn = psycopg2.connect(**self.params)
        except:
            print("Unable to connect to the database.")

    def close(self):
        if self.conn is not None:
            self.conn.close()
        else:
            print("Connection already closed. No action taken.")

We can create a `DbConnection` like this:

In [75]:
conn = DbConnection(params)

Now we want to be able to pull data from the database to use and analyze. We can do this by passing our connection and table name to the pull_data function

In [76]:
def pull_data(conn, tablename):

        """
        Pulls data from the database for a given table and returns a pandas dataframe.
        See https://pklmart.com/postgres_pklm_pklm_prd/relationships.html for a list of tables.
        """
        #Open the connection
        conn.connect()

        with conn.conn.cursor() as cursor:
            try:
                cursor.execute(
                    f"SELECT column_name FROM information_schema.columns where table_name='{tablename}';"
                )
                cols = cursor.fetchall()
                cols = [cols[x][0] for x in range(len(cols))]
                cursor.execute(f"SELECT * FROM pklm_prd.{tablename}")
                data = cursor.fetchall()
                conn.close()
                return pd.DataFrame(data, columns=cols)
            except Exception as inst:
                print(type(inst))
                print(inst.args)
                print(inst)

The tables that exist in the database are:

In [77]:
#| hide
table_names = ["tournament", "match"]

In [78]:
table_names

['tournament', 'match']

In [79]:
match = pull_data(conn, "match")

In [80]:
match.head()

,match_id,tourn_id,consol_ind,team_id_1,team_id_2,maint_dtm,maint_app,create_dtm,create_app
0,M1,T1,N,T1,T2,2022-04-09 03:19:33.840951+00:00,postgres,2022-04-09 03:19:33.840951+00:00,postgres
1,M2,T2,N,T2,T3,2022-05-26 00:45:11.301752+00:00,postgres,2022-05-26 00:45:11.301752+00:00,postgres
2,M5,T5,N,T6,T5,2022-06-28 00:40:22.948360+00:00,postgres,2022-06-28 00:40:22.948360+00:00,postgres
3,M6,T6,N,T5,T7,2022-07-07 23:01:45.921540+00:00,postgres,2022-07-07 23:01:45.921540+00:00,postgres
4,M7,T7,N,T8,T9,2022-07-11 02:40:50.597016+00:00,postgres,2022-07-11 02:40:50.597016+00:00,postgres


In [81]:
#| hide
import nbdev; nbdev.nbdev_export()